In [4]:
!pip install boto3 openai ipython


  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.1/948.1 kB 3.1 MB/s  0:00:00 eta 0:00:01
Using cached anyio-4.10.0-py3-none-any

In [84]:
import os
import boto3
import json
from openai import OpenAI
from datetime import datetime
from IPython.display import clear_output, display, display_markdown, Markdown


In [85]:
# Model Configuration - DeepSeek-V3.1
MODEL_ID = "deepseek.v3-v1:0"  

print(f"✅ Using model: {MODEL_ID}")


✅ Using model: deepseek.v3-v1:0


In [86]:
# Set environment variables to point to Bedrock
# Note: Change the region in the URL to match your preferred region
# Supported regions:
# US: us-east-1, us-west-2
# EU: eu-west-1, eu-west-2, eu-north-1, eu-south-1
# AP: ap-northeast-1, ap-south-1
# SA: sa-east-1
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "ABSKQmVkcm9ja0FQSUtleS15eXJqLWF0LTQwNTY0NTIyMjcyODpHVkRMbWpZQXNRQXlVd2cwc0RmSXpCeWdXY3J5OGVONFpQaDVncXVFVVh1b2k0UXdhbHZiVFYwK2Vycz0="
os.environ["OPENAI_API_KEY"] = "ABSKQmVkcm9ja0FQSUtleS15eXJqLWF0LTQwNTY0NTIyMjcyODpHVkRMbWpZQXNRQXlVd2cwc0RmSXpCeWdXY3J5OGVONFpQaDVncXVFVVh1b2k0UXdhbHZiVFYwK2Vycz0="
os.environ["OPENAI_BASE_URL"] = "https://bedrock-runtime.us-west-2.amazonaws.com/openai/v1"

print("✅ Environment configured for Bedrock!")
print("📍 Using us-west-2 region - change the URL above to use a different region")


✅ Environment configured for Bedrock!
📍 Using us-west-2 region - change the URL above to use a different region


In [87]:
# Initialize both clients
# Note: Change region_name to match your preferred region
# Supported regions:
# US: us-east-1, us-west-2
# EU: eu-west-1, eu-west-2, eu-north-1, eu-south-1
# AP: ap-northeast-1, ap-south-1
# SA: sa-east-1
client = OpenAI()  # For chat completions API
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')  

print("✅ OpenAI client initialized (pointing to Bedrock)")
print(f"✅ Bedrock client initialized in region: {bedrock_client.meta.region_name}")
print("📍 Change region_name above to use a different supported region")


✅ OpenAI client initialized (pointing to Bedrock)
✅ Bedrock client initialized in region: us-west-2
📍 Change region_name above to use a different supported region


In [88]:
# Example 1: Non-thinking mode (quick response)
response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a concise, highly logical assistant."},
        {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
    ],
    temperature=0,
    max_completion_tokens=1000,
    # Non-thinking mode for quick responses

)

# Extract and print the response text
print("🤖 Non-thinking mode response:")
print(response.choices[0].message.content)


🤖 Non-thinking mode response:
The largest city in the southern hemisphere is São Paulo, Brazil. It has a metropolitan population of over 22 million people.


In [89]:
# Example 2: Thinking mode (step-by-step reasoning)
response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that thinks through problems step by step."},
        {"role": "user",   "content": "If a train leaves station A at 60 mph and another leaves station B at 40 mph, and they are 200 miles apart, when will they meet?"}
    ],
    temperature=0,
    max_completion_tokens=2000,
    # reasoning_effort values: low, medium, high
    reasoning_effort='high'  # Thinking mode for complex reasoning
)

# Extract and print the response text
print("🧠 Thinking mode response:")
print(response.choices[0].message.content)


🧠 Thinking mode response:
<reasoning>First, the problem is about two trains moving towards each other from different stations. Train A leaves station A at 60 mph, and train B leaves station B at 40 mph. The distance between the stations is 200 miles. I need to find out when they will meet.

Since they are moving towards each other, their speeds will add up when considering the relative speed. The relative speed is the sum of their speeds because they're approaching each other. So, relative speed = 60 mph + 40 mph = 100 mph.

The distance between them is 200 miles. To find the time it takes for them to meet, I use the formula: time = distance / relative speed.

So, time = 200 miles / 100 mph = 2 hours.

Therefore, they will meet after 2 hours.

I should double-check. Let's think about the distances each train travels in that time.

In 2 hours, train A at 60 mph will travel 60 * 2 = 120 miles.

Train B at 40 mph will travel 40 * 2 = 80 miles.

Together, 120 + 80 = 200 miles, which is the

In [90]:
# Streaming with thinking mode
streaming_response = client.chat.completions.create(
    model=MODEL_ID,                 
    messages=[
        {"role": "system", "content": "You are a helpful assistant that thinks through problems step by step."},
        {"role": "user",   "content": "Explain how photosynthesis works in simple terms."}
    ],
    temperature=0,
    max_completion_tokens=1500,
    reasoning_effort='high',  # Enable thinking mode
    stream=True
)

# Extract and print the response text in real-time.
print("🧠 Streaming thinking mode response:")
for chunk in streaming_response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


🧠 Streaming thinking mode response:
<reasoning>Hmm, the user is asking for a simple explanation of photosynthesis. They</reasoning><reasoning> probably want a clear, easy-to-understand breakdown without too much scientific jargon</reasoning><reasoning>. 

I should start with the basic purpose of photosynthesis—how plants make</reasoning><reasoning> their own food. Then break it down into the key ingredients: sunlight</reasoning><reasoning>, water, and carbon dioxide. 

I can use a simple</reasoning><reasoning> analogy, like a kitchen recipe, to make it relatable. Mention the</reasoning><reasoning> role of chlorophyll and the byproducts (oxygen and sugar) to give</reasoning><reasoning> a complete picture. 

Keep it concise but cover all the essentials:</reasoning><reasoning> what goes in, what happens, and what comes out. Avoid diving</reasoning><reasoning> into complex details like light-dependent reactions or Calvin cycle—that might overwhelm</reasoning><reasoning> the user. 

End wit

In [91]:
def get_weather(location):
    """
    Get current weather for a given location.
    This is a mock function that returns sample weather data for demonstration.
    
    Args:
        location (str): City name or "City, Country" format
        
    Returns:
        dict: Weather information with temperature, condition, and humidity
    """
    # Enhanced mock weather data - in a real application, you'd call a weather API
    weather_data = {
        # Exact matches
        "Paris, France": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "Tokyo, Japan": {"temperature": "25°C", "condition": "Light rain", "humidity": "80%"},
        "New York, USA": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Sydney, Australia": {"temperature": "28°C", "condition": "Clear skies", "humidity": "55%"},
        "London, UK": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"},
        
        # City-only matches for more flexible parsing
        "Paris": {"temperature": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "Tokyo": {"temperature": "25°C", "condition": "Light rain", "humidity": "80%"},
        "New York": {"temperature": "18°C", "condition": "Sunny", "humidity": "45%"},
        "Sydney": {"temperature": "28°C", "condition": "Clear skies", "humidity": "55%"},
        "London": {"temperature": "15°C", "condition": "Overcast", "humidity": "70%"}
    }
    
    return weather_data.get(location, {
        "temperature": "23°C", 
        "condition": "Pleasant", 
        "humidity": "60%"
    })

# Define the function schema for OpenAI SDK
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature and weather conditions for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country, e.g. 'Paris, France'"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        }
    }
}]

print("✅ Weather function and tools configuration ready!")


✅ Weather function and tools configuration ready!


In [92]:
def chat_with_functions(client, model, messages, tools, max_iterations=3, reasoning_effort='high'):
    """
    Chat with function calling support using OpenAI SDK format.
    
    Args:
        client: OpenAI client instance
        model: Model ID to use
        messages: List of conversation messages
        tools: List of available tools/functions
        max_iterations: Maximum number of function call iterations
        reasoning_effort: Whether to use thinking mode for enhanced reasoning
        
    Returns:
        Final assistant message
    """
    
    for iteration in range(max_iterations):
        print(f"🔄 Iteration {iteration + 1}")
        
        # Make request with tools
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            reasoning_effort=reasoning_effort  # Use thinking mode for enhanced tool calling
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Check if the model wants to call functions
        if assistant_message.tool_calls:
            print(f"🔧 Model requested {len(assistant_message.tool_calls)} function call(s)")
            
            # Process each function call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🔧 Calling function: {function_name}")
                print(f"🔧 Arguments: {function_args}")
                
                # Call the actual function
                if function_name == "get_weather":
                    function_result = get_weather(function_args["location"])
                    print(f"🔧 Function result: {function_result}")
                else:
                    function_result = {"error": f"Unknown function: {function_name}"}
                
                # Add function result to conversation
                function_message = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": json.dumps(function_result)
                }
                messages.append(function_message)
                
        else:
            # No more function calls, return final response
            print("✅ No function calls requested, conversation complete")
            return assistant_message
    
    print("⚠️ Maximum iterations reached")
    return assistant_message

print("✅ Enhanced function calling handler ready!")


✅ Enhanced function calling handler ready!


In [93]:
# Test enhanced function calling with thinking mode
weather_questions = [
    "What's the weather like in Paris today?",
    "Can you tell me the temperature in Tokyo?",
    "How's the weather in Sydney, Australia?",
    "What are the conditions like in New York?"
]

print("🌤️ Testing Enhanced Function Calling with DeepSeek-V3.1")
print("=" * 60)

for i, question in enumerate(weather_questions, 1):
    print(f"\n📝 Test {i}: {question}")
    print("-" * 40)
    
    try:
        # Create conversation messages
        messages = [
            {"role": "system", "content": "You are a helpful weather assistant. When users ask about weather, temperature, or weather conditions for any location, you MUST call the get_weather function to get current information. Always use the function when weather information is requested."},
            {"role": "user", "content": question}
        ]
        
        # Test function calling directly with tool_choice="required"
        print("🔧 Testing function call...")
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            tools=tools,
            tool_choice="required",  # Force function calling
            reasoning_effort='high'
        )
        
        if response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            print(f"✅ Function called successfully: {tool_call.function.name}")
            print(f"🔧 Arguments: {tool_call.function.arguments}")
            
            # Execute the function and show result
            import json
            args = json.loads(tool_call.function.arguments)
            function_result = get_weather(args.get("location", "Unknown"))
            print(f"🔧 Function result: {function_result}")
            
            # Generate a dummy response based on the function result
            location = args.get("location", "Unknown")
            temp = function_result.get("temperature", "Unknown")
            condition = function_result.get("condition", "Unknown")
            humidity = function_result.get("humidity", "Unknown")
            
            dummy_response = f"Based on current weather data for {location}: Temperature is {temp} with {condition.lower()} conditions and {humidity} humidity. Great for planning your day!"
            
            print("🤖 Generated weather report:")
            print(dummy_response)
            print("✅ Function calling demonstration complete!")
            
        else:
            print("❌ No function call made")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print()


🌤️ Testing Enhanced Function Calling with DeepSeek-V3.1

📝 Test 1: What's the weather like in Paris today?
----------------------------------------
🔧 Testing function call...
✅ Function called successfully: get_weather
🔧 Arguments: {"location": "Paris"}
🔧 Function result: {'temperature': '22°C', 'condition': 'Partly cloudy', 'humidity': '65%'}
🤖 Generated weather report:
Based on current weather data for Paris: Temperature is 22°C with partly cloudy conditions and 65% humidity. Great for planning your day!
✅ Function calling demonstration complete!


📝 Test 2: Can you tell me the temperature in Tokyo?
----------------------------------------
🔧 Testing function call...
✅ Function called successfully: get_weather
🔧 Arguments: {"location": "Tokyo"}
🔧 Function result: {'temperature': '25°C', 'condition': 'Light rain', 'humidity': '80%'}
🤖 Generated weather report:
Based on current weather data for Tokyo: Temperature is 25°C with light rain conditions and 80% humidity. Great for planning yo

In [94]:
# Configure region for Bedrock client
# Supported regions:
# US: us-east-1, us-west-2
# EU: eu-west-1, eu-west-2, eu-north-1, eu-south-1
# AP: ap-northeast-1, ap-south-1
# SA: sa-east-1
region = "us-west-2"

if region is None:
    target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
else:
    target_region = "us-west-2"

bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)
print(f"📍 Using region: {target_region} - change the region variable above to use a different supported region")


📍 Using region: us-west-2 - change the region variable above to use a different supported region


In [95]:
def invoke_model(body, model_id, accept, content_type):
    """
    Invokes Amazon bedrock model to run an inference
    using the input provided in the request body.
    
    Args:
        body (dict): The invokation body to send to bedrock
        model_id (str): the model to query
        accept (str): input accept type
        content_type (str): content type
    Returns:
        Inference response from the model.
    """

    try:
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body), 
            modelId=model_id, 
            accept=accept, 
            contentType=content_type
        )

        return response

    except Exception as e:
        print(f"Couldn't invoke {model_id}")
        raise e


In [96]:
# Example with thinking mode enabled
messages = [
    {"role": "system", "content": "You are a concise, highly logical assistant."},
    {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
]

body = {
    "messages": messages,
    "temperature": 0,
    "max_completion_tokens": 1000,
    "reasoning_effort": "high"  # Enable thinking mode for DeepSeek-V3.1
}

accept = "application/json"
contentType = "application/json"

response = invoke_model(body, MODEL_ID, accept, contentType)
response_body = json.loads(response.get("body").read())

print("🧠 Thinking mode response:")
print(response_body['choices'][0]['message']['content'])


🧠 Thinking mode response:
<reasoning>Hmm, the user is asking for the largest city in the southern hemisphere. This is a straightforward factual question, so I should provide a clear and concise answer. 

I recall that São Paulo, Brazil, is often cited as the largest city in the southern hemisphere by population. I should confirm this and include the population figure to make the response more informative. 

Since the user didn’t specify any other criteria like metropolitan area or land area, I’ll stick to population as the default metric. I’ll also mention the country to give context. 

No need to overcomplicate it—just state the answer directly with the supporting data.</reasoning>The largest city in the southern hemisphere by population is São Paulo, Brazil, with over 12 million residents in the city proper and more than 22 million in its metropolitan area.


In [97]:
# Streaming with thinking mode
messages = [
    {"role": "system", "content": "You are a concise, highly logical assistant."},
    {"role": "user",   "content": "What is the largest city in the southern hemisphere?"}
]

body = {
    "messages": messages,
    "temperature": 0,
    "max_completion_tokens": 1000,
    "reasoning_effort": "high"  # Enable thinking mode for streaming
}

accept = "application/json"
contentType = "application/json"

start_time = datetime.now()

response = bedrock_runtime.invoke_model_with_response_stream(
    body=json.dumps(body), modelId=MODEL_ID, accept=accept, contentType=contentType
)
chunk_count = 0
time_to_first_token = None

# Process the response stream
stream = response.get("body")
if stream:
    print("🧠 Streaming thinking mode response:")
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            # Print the response chunk
            chunk_json = json.loads(chunk.get("bytes").decode())
            content_block_delta = chunk_json.get("choices")[0]["delta"].get("content")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = datetime.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")

                chunk_count += 1
                print(content_block_delta, end="")
    print(f"\nTotal chunks: {chunk_count}")
else:
    print("No response stream received.")


🧠 Streaming thinking mode response:
Time to first token: 0:00:00.911073
<reasoning>Hmm, the user is asking for the largest city in the southern hemisphere</reasoning><reasoning>. This is a straightforward factual question, so I should provide a clear</reasoning><reasoning> and concise answer. 

I recall that São Paulo, Brazil, is</reasoning><reasoning> often cited as the largest city in the southern hemisphere by population. I</reasoning><reasoning> should confirm this and include the population figure to make the response more</reasoning><reasoning> informative. 

Since the question is simple, I don’t need</reasoning><reasoning> to overcomplicate it—just state the city, its location,</reasoning><reasoning> and the population. No need for extra details unless the user asks for</reasoning><reasoning> them.</reasoning>The largest city in the southern hemisphere by population is São Paulo, Brazil, with over 12 million residents in the city proper and more than 22 million in its metropolit

In [100]:
# Converse API with thinking mode
response = bedrock_client.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "How far from earth is the moon?"}]
        }
    ],
    system=[{"text": "You are a concise, highly logical assistant."}],
    inferenceConfig={
        "temperature": 0,
        "maxTokens": 1000
    },
    additionalModelRequestFields={
        "reasoning_effort": "high"  # Enable thinking mode for DeepSeek-V3.1
    }
)

# Message dict
print(f"📝 Message dict:")
print(response['output']['message']['content'])

# Reasoning trace (if available)
if 'reasoningContent' in response['output']['message']['content'][0]:
    print(f"📝 Reasoning trace:")
    print(response['output']['message']['content'][0]['reasoningContent']['reasoningText']['text'])

# Final response
print(f"📝 Final response:")
print(response['output']['message']['content'][1]['text'])


📝 Message dict:
[{'reasoningContent': {'reasoningText': {'text': 'Hmm, the user is asking about the distance between Earth and the Moon. This is a straightforward factual question with a well-established answer. \n\nI recall the average distance is about 384,400 km, but it’s worth mentioning that the Moon’s orbit is elliptical, so the actual distance varies. The closest and farthest points might add useful context. \n\nI should keep it concise but include both metric and imperial units since the user didn’t specify a preference. The response should be clear and immediately helpful without unnecessary details. \n\nThe tone should be neutral and informative, matching the user’s straightforward query. No need for fluff or elaboration beyond the key points.'}}}, {'text': "The average distance from Earth to the Moon is approximately **384,400 kilometers (238,855 miles)**. This distance varies slightly due to the Moon's elliptical orbit, ranging from about 363,300 km (225,700 mi) at its clos

In [102]:
def bedrock_model_converse_stream(client, system_prompt, user_prompt, max_tokens=1000, temperature=0, reasoning_effort='high'):
    response = ""
    response = client.converse_stream(
        modelId=MODEL_ID,
        messages=[  
            {
                "role": "user",
                "content": [
                    {
                        "text": user_prompt
                    }
                ]
            },                        
        ],
        system=[{"text": system_prompt}],
        inferenceConfig={
            "temperature": temperature,
            "maxTokens": max_tokens
        },
        additionalModelRequestFields={
            "reasoning_effort": reasoning_effort  # Enable thinking mode for DeepSeek-V3.1
        }
    )
    # Extract and print the response text in real-time.
    for event in response['stream']:
        if 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']
            if chunk['delta'].get('reasoningContent', None):
                print(chunk['delta']['reasoningContent']['text'], end="")
            if chunk['delta'].get('text', None):
                print(chunk['delta']['text'], end="")
    return


In [103]:
# Define input prompts
system_prompt = "You are a concise, highly logical assistant."
user_prompt = "How far from earth is the moon?"

# Invoke model through Converse API with streaming and thinking mode
print("🧠 Streaming with thinking mode:")
bedrock_model_converse_stream(bedrock_client, system_prompt, user_prompt, reasoning_effort='high')


🧠 Streaming with thinking mode:
Hmm, the user is asking about the distance between Earth and the Moon. This is a straightforward factual question with a well-established answer. 

I recall the average distance is about 384,400 km, but since the Moon's orbit is elliptical, the actual distance varies. I should mention both the average and the range to be precise. 

The user might also appreciate a relatable comparison, so I can add that it's roughly 30 Earth-widths away. No need to overcomplicate it—just the key numbers and a simple analogy. 

The response should be concise but cover the basics: average distance, variation, and a tangible comparison. No extra fluff needed.The average distance from Earth to the Moon is approximately **384,400 kilometers (238,855 miles)**. This distance varies because the Moon follows an elliptical orbit, ranging from about 363,300 km (225,700 mi) at its closest (perigee) to 405,500 km (252,000 mi) at its farthest (apogee). For perspective, this is roughly